In [74]:
import glob
import os
import random
from pathlib import Path
import cv2
import numpy as np

import sys
from os.path import abspath, join, dirname
base_path = abspath('./../training/model/yolov7/')
sys.path.append(base_path)

from utils.datasets import letterbox

img_formats = ['bmp', 'jpg', 'jpeg', 'png', 'tif', 'tiff', 'dng', 'webp', 'mpo']  # acceptable image suffixes

class LoadImages:  
    def __init__(self, path, img_size=640, stride=32):
        
        # path is directing to the json file with labels
        image_ids = []
        with open(path, 'r') as f:
            data = json.load(f)
            for image_id, info in data.items():
                image_ids.append(image_id)

        image_ids_set = set(image_ids)
        print(f'found {len(image_ids)} image IDs in {path}')
        print(f'ended up with {len(image_ids_set)} IDs after converting to a set')
        
        images = []
        for image_id in image_ids_set:
            image_path = join(dirname(dirname(path)), 'like', image_id + '.jpg')
            images.append(image_path)
        
        print(f'list image path length = {len(images)}')
        ni = len(images)

        self.img_size = img_size
        self.stride = stride
        self.files = images
        self.nf = ni  # number of files
        self.video_flag = False
        self.mode = 'image'
        self.cap = None
        assert self.nf > 0, f'No images found in {p}. ' \
                            f'Supported formats are:\nimages: {img_formats}'

    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        if self.count == self.nf:
            raise StopIteration
        path = self.files[self.count]

        # Read image
        self.count += 1
        img0 = cv2.imread(path)  # BGR
        assert img0 is not None, 'Image Not Found ' + path

        # Padded resize
        img = letterbox(img0, self.img_size, stride=self.stride)[0]

        # Convert
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        return path, img, img0, self.cap

In [104]:
def hagrid_xywh_to_xyxy(bboxes, img_shape):
    """
    Convert bounding boxes from Hagrid format (xywh) to (xyxy).

    Args:
        bboxes (list of list): List of bounding boxes in the format
                               [[top left X, top left Y, width, height], ...].
        img_shape (tuple): Shape of the image as (height, width, channels).

    Returns:
        list of list: List of converted bounding boxes in the format
                       [[x_min, y_min, x_max, y_max], ...].
    """
    
    height, width, _ = img_shape
    xyxy_bboxes = []
    
    for bbox in bboxes:
        x, y, w, h = bbox
        
        # Convert normalized values back to pixel values
        x_min = x * width
        y_min = y * height
        x_max = (x + w) * width
        y_max = (y + h) * height
        
        xyxy_bboxes.append([x_min, y_min, x_max, y_max])
    
    return xyxy_bboxes

In [117]:
from pathlib import Path
import torch
# import torch.backends.cudnn as cudnn
from numpy import random
import sys
from os.path import abspath
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

# Add the new path to the system path
base_path = abspath('./../training/model/yolov7/')
sys.path.append(base_path)

from models.experimental import attempt_load
# from utils.datasets import LoadImages
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.torch_utils import select_device, TracedModel
from utils.plots import plot_one_box


def detect(opt):
    source, weights, view_img, save_txt, imgsz, trace = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, not opt.no_trace

    # Initialize
    device = select_device(opt.device)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size

    # if trace:
    #     model = TracedModel(model, device, opt.img_size)

    if half:
        model.half()  # to FP16

    # Set Dataloader
    dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Get all gestures bbxs info
    hagrid_annotations = {}
    with open(os.path.join(source), 'r') as f:
        data = json.load(f)

        # Check if images in the label file are in the test set
        for image_id, info in data.items():
            hagrid_annotations[image_id] = info
    print(f'hagrid_annotations dictionary length = {len(hagrid_annotations)}')
    
    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    for path, img, im0s, vid_cap in tqdm(dataset):
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment=opt.augment)[0]

        # Inference
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            print(f"i {i}")
            p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path        p.name=img.jpg
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} person{'s' * (n > 1)}, "  # add to string
                    print(s)
                
                # print(det)
                
                best_bbox = None
                highest_conf = 0

                # Iterate through the detections
                for *xyxy, conf, _ in det:
                    conf = conf.item()  # Convert confidence tensor to float

                    # Update if the current confidence is higher than the highest found
                    if conf > highest_conf:
                        highest_conf = conf
                        best_bbox = xyxy
                # print(best_bbox)
                
            else:
                print("no human detected")
                print(im0.shape)
                best_bbox = torch.tensor([0, 0, im0.shape[1], im0.shape[0]], device='cuda:0')
                print(best_bbox)
        
        # Get ground truth gestures bbxs in xyxy format
        gestures_info = hagrid_annotations[p.stem]
        gestures_bbxs = hagrid_xywh_to_xyxy(gestures_info['bboxes'], im0.shape)
        
        print(f'Image {p.name}') # p - path to image
        print(f'image loaded, shape: {im0.shape}') # im0 - original image
        print(f'human detected, bbox: {best_bbox}') # best bbox - human bbx for cropping in xyxy
        print(f'ground truth gestures loaded, bboxes: {gestures_bbxs}') # hand_bbxs - hands bbxs groud truth in xyxy
        
#     plot_one_box(best_bbox, im0, label='Human', color=(255, 0, 0), line_thickness=3)
#     for gesture_bbox in gestures_bbxs:
#         plot_one_box(gesture_bbox, im0, label='Gesture', color=(0, 255, 0), line_thickness=3)
#         # cv2.imshow(str(p), im0)
#         # cv2.waitKey(1)  # 1 millisecond
        
        
#     plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
#     plt.show()
#     plt.pause(0.001)  # Use a small value for quick transitions
#     plt.clf() 
        

In [118]:
import argparse

opt = argparse.Namespace(
    weights=['./../training/runs/train/MIAP_person_detection/weights/best.pt'],
    source="./../datasets/HaGRID_test/val/like.json",
    img_size=640,
    conf_thres=0.317,
    iou_thres=0.45,
    device='0',
    view_img=False,
    save_txt=False,
    save_conf=False,
    nosave=False,
    classes=None,
    agnostic_nms=False,
    augment=False,
    update=False,
    project='runs/detect',
    name='reducing_detect_func',
    exist_ok=False,
    no_trace=False
)

detect(opt)

Fusing layers... 
IDetect.fuse
found 100 image IDs in ./../datasets/HaGRID_test/val/like.json
ended up with 100 IDs after converting to a set
list image path length = 100
hagrid_annotations dictionary length = 100


2it [00:00, 11.83it/s]

i 0
2 persons, 
Image 002cc8a1-7a8b-4b2f-bf0f-7f74548c37b4.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(224., device='cuda:0'), tensor(1266., device='cuda:0'), tensor(1030., device='cuda:0'), tensor(1920., device='cuda:0')]
ground truth gestures loaded, bboxes: [[381.702096, 1482.3213696, 532.6340112, 1697.596128]]
i 0
1 person, 
Image 0082629f-d111-424f-8a65-ae8314a7f5ec.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(0., device='cuda:0'), tensor(712., device='cuda:0'), tensor(922., device='cuda:0'), tensor(1920., device='cuda:0')]
ground truth gestures loaded, bboxes: [[332.163864, 1002.2755775999999, 582.5210976, 1366.3686911999998]]
i 0
1 person, 
Image 00c15276-437f-49ef-84ac-80cab0a01f26.jpg
image loaded, shape: (640, 480, 3)
human detected, bbox: [tensor(4., device='cuda:0'), tensor(4., device='cuda:0'), tensor(478., device='cuda:0'), tensor(638., device='cuda:0')]
ground truth gestures loaded, bboxes: [[123.9533472, 229.656966400

7it [00:00, 13.59it/s]

i 0
2 persons, 
Image 000f0e41-9f04-4d07-b328-2be30e3563ef.jpg
image loaded, shape: (1080, 1920, 3)
human detected, bbox: [tensor(646., device='cuda:0'), tensor(307., device='cuda:0'), tensor(1269., device='cuda:0'), tensor(979., device='cuda:0')]
ground truth gestures loaded, bboxes: [[1006.6659840000001, 445.83298560000003, 1154.6716416, 654.0506352]]
i 0
1 person, 
Image 005762c1-ddc8-4ea3-becd-05f3ec326b03.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(343., device='cuda:0'), tensor(730., device='cuda:0'), tensor(846., device='cuda:0'), tensor(1888., device='cuda:0')]
ground truth gestures loaded, bboxes: [[347.9302224, 1402.051488, 421.1425872, 1535.5348608], [568.5979968, 934.9836096, 662.0827824, 1074.2397312]]
i 0
1 person, 
Image 00277cca-b999-4527-bba3-9461fd2eb45b.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(393., device='cuda:0'), tensor(400., device='cuda:0'), tensor(965., device='cuda:0'), tensor(1662., device='cuda:0')]
g

...

i 0
1 person, 
Image 00b6c2e6-f283-4f7a-ab51-0e4e2da05b53.jpg
image loaded, shape: (1920, 1080, 3)
human detected, bbox: [tensor(252., device='cuda:0'), tensor(472., device='cuda:0'), tensor(914., device='cuda:0'), tensor(1914., device='cuda:0')]
ground truth gestures loaded, bboxes: [[440.61947999999995, 982.6191935999999, 595.5329628, 1190.515872]]
i 0
1 person, 
Image 004881c0-533c-46f1-b7a8-e7633dd41f68.jpg
image loaded, shape: (480, 640, 3)
human detected, bbox: [tensor(11., device='cuda:0'), tensor(57., device='cuda:0'), tensor(585., device='cuda:0'), tensor(479., device='cuda:0')]
ground truth gestures loaded, bboxes: [[184.94428159999998, 212.64363360000002, 297.2277888, 380.8932048]]
i 0
1 person, 
Image 00aa414d-2b1e-4504-8ca8-b1da1ebb7538.jpg
image loaded, shape: (1440, 1920, 3)
human detected, bbox: [tensor(633., device='cuda:0'), tensor(146., device='cuda:0'), tensor(1724., device='cuda:0'), tensor(1438., device='cuda:0')]
ground truth gestures loaded, bboxes: [[1089.13355

98it [00:05, 21.13it/s]

i 0
1 person, 
Image 000484ab-5fd0-49b8-9253-23a22b71d7b1.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(282., device='cuda:0'), tensor(444., device='cuda:0'), tensor(1430., device='cuda:0'), tensor(1917., device='cuda:0')]
ground truth gestures loaded, bboxes: [[501.62999040000005, 1657.4670336, 875.1230064000001, 1906.5631872], [837.903888, 859.3820352, 1075.1846256000001, 1172.0039424]]
i 0
1 person, 
Image 0030c39d-f220-4be5-9ae5-fe176c17a821.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(317., device='cuda:0'), tensor(530., device='cuda:0'), tensor(1420., device='cuda:0'), tensor(1916., device='cuda:0')]
ground truth gestures loaded, bboxes: [[756.5215968, 836.610816, 936.5750495999999, 1107.1706304]]
i 0
1 person, 
Image 00d56515-03a6-45a9-853e-5ae7ea46e374.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(206., device='cuda:0'), tensor(508., device='cuda:0'), tensor(1108., device='cuda:0'), tensor(1917., device

100it [00:06, 16.29it/s]


i 0
1 person, 
Image 001c5c43-c9fe-4766-b791-d6bf309c7f95.jpg
image loaded, shape: (1920, 1440, 3)
human detected, bbox: [tensor(19., device='cuda:0'), tensor(441., device='cuda:0'), tensor(1401., device='cuda:0'), tensor(1917., device='cuda:0')]
ground truth gestures loaded, bboxes: [[461.4044688, 1131.9921408, 769.1532768, 1596.0994176]]


In [ ]:
from utils.general import xyxy2xywh

# prep HaGRID_test

In [27]:
import os
import json
import shutil

# Paths
original_dataset_path = './../datasets/HaGRID/'
test_dataset_path = './../datasets/HaGRID_test/'

# Load the 100 test images
test_images = [f for f in os.listdir(os.path.join(test_dataset_path, 'like')) if f.endswith('.jpg')]

# Create a set for easier matching
test_image_set = set(os.path.splitext(f)[0] for f in test_images)

In [28]:
len(test_images)

100

In [29]:
len(test_image_set)

100

In [35]:
# test_image_set

In [37]:
original_dataset_path = './../datasets/HaGRID/'
test_dataset_path = './../datasets/HaGRID_test/'

In [53]:
# Initialize an empty list for the labels
matched_labels = {}

# Iterate through label files in the original dataset
for split in ['train', 'val', 'test']:
    label_path = os.path.join(original_dataset_path, split)
    with open(os.path.join(label_path, "like.json"), 'r') as f:
        data = json.load(f)

        # Check if images in the label file are in the test set
        for image_id, info in data.items():
            if image_id in test_image_set:
                matched_labels[image_id] = info

In [52]:
sample_keys=random.sample(list(matched_labels.keys()), min(10, len(matched_labels)))
for image_id in sample_keys:
    print(f'{{{image_id} : {matched_labels[image_id]}}}')

{002d020f-7843-4018-a085-484e7319b00a : {'bboxes': [[0.3666225, 0.603495, 0.11481563, 0.2389275]], 'user_id': '25bc760d508911e2c3453c3dd593092a0bf1b83c01817182eea3899fa76964c2', 'labels': ['like']}}
{00818540-8153-41d3-aba7-2369eaea25b9 : {'bboxes': [[0.42623638, 0.4084392, 0.07486643, 0.18070222]], 'user_id': 'a1b0bb3f79eb35269ae7e804eea8557fb1c7e97259cb8b5d98bba7fc870d91da', 'labels': ['like']}}
{00bf711f-e2cc-4cb5-b0ac-573198cd6455 : {'bboxes': [[0.32369747, 0.54091418, 0.11421507, 0.12234949], [0.59147623, 0.87560417, 0.13760348, 0.09844989]], 'user_id': '22ce972ece766dc47a240644048fb8399e4e69075f225c634bfaacb2a84e7423', 'labels': ['like', 'no_gesture']}}
{0069d29e-4e71-433b-96e1-09657ca3d7eb : {'bboxes': [[0.42199823, 0.3240404, 0.08642329, 0.22055762]], 'user_id': '31ef5349bc0bdc95cb2dcede7fc6bdce4c99b4ea4e70bb495f50082ec1f32afe', 'labels': ['like']}}
{009b4a76-9223-4801-a7dc-31d45cab3df9 : {'bboxes': [[0.30040455, 0.6490744, 0.09700108, 0.11034548]], 'user_id': 'dd7f74b0a40f86ca

In [54]:
output_path = os.path.join(test_dataset_path, 'val', 'like.json')
with open(output_path, 'w') as outfile:
    json.dump(matched_labels, outfile, indent=4)

print(f"\nMatched labels saved to {output_path}.")


Matched labels saved to ./../datasets/HaGRID_test/val/like.json.


In [58]:
test_labels = []
with open(output_path, 'r') as f:
    data = json.load(f)
    for image_id, info in data.items():
        test_labels.append(image_id)

test_label_set = set(test_labels)

In [57]:
len(test_labels)

100

In [61]:
len(test_label_set)

100

In [62]:
test_image_set == test_label_set

True